## INTRODUCTION.

This Notebook works to for a prediction of various disrders based on the user signs.

The output also includes reccommendations for treatment.

For this we use the data in Data_Mental_Health.csv with various mental disorder signs and their treatment reccommendations.

The model we use is a Sentence Transformer that reads various signs and assigns various disorder probabilities, then for the output we consider that with the highest probability.

## Import Necessary Libraries.

In [ ]:
#Import necessary libraries.
import pandas as pd

# Load Excel

df_signs = pd.read_excel(" Final mental health data .xlsx", sheet_name="Signs")
df_recs = pd.read_excel("Final mental health data .xlsx", sheet_name="Reccomendatons")

grouped = df_signs.groupby('Disorder')['Signs'].apply(lambda signs: ' '.join(signs.astype(str))).reset_index()

grouped

,Disorder,Signs
0,Anxiety,A feeling of dread or fearing the worst Feelin...
1,Attention Deficit Hyperactivity Disorder(ADHD),Short attention span Making careless mistakes ...
2,Autism,Difficulty recorgnising and understanding othe...
3,Bipolar disorder,Feeling sad and hopeless Lack of energy and ap...
4,Dementia,"memory loss difficulty with concentrating, pla..."
5,Depression,"empty and numb guilty or worthless sad, upset ..."
6,Loneliness,BeingLGBTQIA+ Being widowed Being single Being...
7,Menopause,"hot flushes – short, sudden bursts of heat tha..."
8,OCD,"Fear of contamination (e.g., germs, dirt) Fear..."
9,PTSD,Feeling shocked and distressed after a traumat...


In [15]:
# Create the dictionary from the DataFrame
signs_dict = dict(zip(grouped['Disorder'], grouped['Signs']))
signs_dict

{'Anxiety': 'A feeling of dread or fearing the worst Feeling on edge or panicky Difficulties concerntrating Irritability Feeling detatched from yourself or the world around you Restlessness Feeeling dizzy or light-headedness Wobbly legs Pins and needles in your hands and feet Shortness of breath and hyperventilating Heart palpitations Nausea Needing the toilet more or less often Sweating Sleeping problems Panic attacks',
 'Attention Deficit Hyperactivity Disorder(ADHD)': 'Short attention span Making careless mistakes Being forgetul and losing things Being unable to stick to boring tasks Constantly changing activity Dificulty organising tasks Fidgetting or being unable to stay still Lots of physical movements or talking Being unable to wait for your time Acting impulsively or recklessly Interupting conversations Daydreaming and zoning out Poor listening skills Risk taking eg dangerous driving Impatience, mood swings and irritability',
 'Autism': "Difficulty recorgnising and understandin

## MODELLING.

In [ ]:
#Modeling
# Import necessary libraries for modeling
from sentence_transformers import SentenceTransformer, util

# Load Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode disorder descriptions
disorder_embeddings = {
    disorder: model.encode(sign_text)
    for disorder, sign_text in signs_dict.items()
}


/Users/faithkamande/miniconda3/envs/learn-env/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Define the prediction function.

In [17]:
# Define prediction function
def predict_disorder_input(user_input, top_k=3):
    print("\n Predicted Disorders & Similarities:\n")

    input_embed = model.encode(user_input)
    similarities = {}

    for disorder, embed in disorder_embeddings.items():
        score = util.cos_sim(input_embed, embed).item()
        similarities[disorder] = score

    # Sort by score
    sorted_disorders = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:top_k]

    for disorder, score in sorted_disorders:
        print(f"- {disorder}: {score:.2f}")

    # Get top disorder
    top_disorder = sorted_disorders[0][0]
    print(f"\n*** Top Concern: {top_disorder}")

    # 7. Fetch recommendations
    rec_row = df_recs[df_recs['Disorder'].str.lower() == top_disorder.lower()]
    if not rec_row.empty:
      print("\n* Self-care:")
      for i, val in enumerate(rec_row['Reccomendations; Self'].dropna().values[:5], 1):
        print(f"  - Option {i}: {val}")

    print("\n* Professional Help:")
    for i, val in enumerate(rec_row['Reccomendation 2; Proffesional'].dropna().values[:3], 1):
        print(f"  - Option {i}: {val}")

    print("\n* Other Recommendations:")
    for i, val in enumerate(rec_row['Other Reccomendation'].dropna().values[:2], 1):
        print(f"  - Option {i}: {val}")
    else:
        print(" No recommendations found.")


In [18]:
predict_disorder_input("I am bored, I am restless, I cannot concentrate ")



 Predicted Disorders & Similarities:

- Bipolar disorder: 0.41
- Attention Deficit Hyperactivity Disorder(ADHD): 0.41
- Depression: 0.39

*** Top Concern: Bipolar disorder

* Self-care:
  - Option 1: Track moods using journals or apps to identify patterns and triggers

* Professional Help:
  - Option 1: Visit your therapist for CBT(cognitive behavioural threapy)

* Other Recommendations:
 No recommendations found.


## Save the model.

In [19]:
# Save the model and embeddings
import pickle

nlp_bundle = {
    "model": model,
    "embeddings": disorder_embeddings,
    "signs_dict": signs_dict,
    "recommendations": df_recs
}

with open("model_nlp.pkl", "wb") as f:
    pickle.dump(nlp_bundle, f)
